In [2]:
using LinearAlgebra
using Combinatorics
using Statistics
using PyPlot
using BenchmarkTools
using Profile

const X = [0, 1, 1, 0]
const Y = [0, 1im, -1im, 0]
const Z = [1, 0, 0, -1]
const II = [1, 0, 0, 1];

In [7]:
function n_choose_k(n, k)
    if k==0
        return [Bool[0 for _ in 1:n]]
    elseif k==n
        return [Bool[1 for _ in 1:n]]
    else
        a = n_choose_k(n-1, k)
        b = n_choose_k(n-1, k-1)
        return [
            [Bool[0, i...] for i in a]..., 
            [Bool[1, i...] for i in b]...
        ]
    end
end


function charge_q_indices(q)
    indices = Int[]
    
    tmp = n_choose_k(4, q)
    for a in tmp
        idx = sum([2^(i-1) * a[i] for i in 1:n]) + 1
        push!(indices, idx)
    end
    
    tmp = n_choose_k(4, 4-q)
    for a in tmp
        idx = sum([2^(i-1) * a[i] for i in 1:n]) + 1
        push!(indices, idx)
    end
    
    sort!(indices)
    return indices
end


# """
# Hilbert space:
# 1 \otimes 1bar \otimes 2 \otimes 2bar
# """
function charged_paired_state(n, perm, q1, q2)
    d = 2^n
    result = zeros(Int, d^4)
    indices1 = charge_q_indices(q1)
    indices2 = charge_q_indices(q2)
    for a1 in indices1, a2 in indices2
        if perm==false
            idx = (a1-1) * d^0 + (a1-1) * d^1 + (a2-1) * d^2 + (a2-1) * d^3
        else
            idx = (a1-1) * d^0 + (a2-1) * d^1 + (a2-1) * d^2 + (a1-1) * d^3
        end
        result[idx + 1] += 1
    end
    return result
end


function z2gauge_2nd_frame_op()
    d = 2^4
    part1 = zeros(Float32, d^4 * d^4)
    part2 = zeros(Float32, d^4 * d^4)
    
    for q1 in 0:3, q2 in 0:3
        if q1==q2
            continue
        end
        tmp1 = charged_paired_state(n, false, q1, q2)
        tmp2 = charged_paired_state(n, true, q1, q2)
        part1 += (kron(tmp1, tmp1) + kron(tmp2, tmp2)) / (binomial(n, q1) * binomial(n, q2))
    end
    
    for q in 0:n
        dq = binomial(n, q)
        tmp1 = charged_paired_state(n, false, q, q)
        tmp2 = charged_paired_state(n, true, q, q)
        if dq==1
            part2 += kron(tmp1, tmp1)
        else
            part2 += (kron(tmp1, tmp1) + kron(tmp2, tmp2)) / (dq^2 - 1)
            part2 -= (kron(tmp1, tmp2) + kron(tmp2, tmp1)) / ((dq^2 - 1) * dq)
        end
    end
    result = part1 + part2
    result = reshape(result, d^4, d^4)
    return result
end

z2gauge_2nd_frame_op (generic function with 1 method)

In [8]:
q = z2gauge_2nd_frame_op()

LoadError: OutOfMemoryError()